In [5]:
# comment out to run on drive + colab
!pip install transformers_interpret



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [7]:
# from google.colab import drive

#drive.mount('/content/drive/')

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [9]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00


In [10]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import MultiLabelClassificationExplainer
import torch
from transformers import pipeline
from datasets import load_from_disk

import warnings
warnings.filterwarnings('ignore')

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# comment out the filepath to run on drive + colab
filepath = "../models/scenario_level/anomic_conduction_control"
# filepath = "/content/drive/Shareddrives/AphasiaProject/models/scenario_level/anomic_conduction_control"

model = AutoModelForSequenceClassification.from_pretrained(filepath+"/model")
tokenizer = AutoTokenizer.from_pretrained(filepath+"/tokenizer", padding=True,
                                          truncation=True, return_tensors="pt",
                                          add_special_tokens=True, max_length=512)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == torch.device("cuda"):
    model.to(device)
pipe = pipeline("text-classification", model=filepath+"/model", tokenizer=filepath+"/tokenizer", device=device, truncation=True, padding=True)
cls_explainer = MultiLabelClassificationExplainer(model, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
model.config

DistilBertConfig {
  "_name_or_path": "/content/drive/Shareddrives/AphasiaProject/models/anomic_conduction_control/model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "CONTROL",
    "1": "ANOMIC",
    "2": "CONDUCTION"
  },
  "initializer_range": 0.02,
  "label2id": {
    "ANOMIC": 1,
    "CONDUCTION": 2,
    "CONTROL": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.38.2",
  "vocab_size": 30524
}

In [15]:
model.config.id2label

{0: 'CONTROL', 1: 'ANOMIC', 2: 'CONDUCTION'}

In [16]:
def interpret(data):
    for text, label in zip(list(data["new_preprocessed_text"]), list(data["label"])):
        if device == torch.device("cuda"):
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True, max_length=512).input_ids.to(device)
            # tranformers interpret gives error for input of 512 tokens
            if len(inputs[0]) != 512:
                with torch.no_grad():
                    logits = model(inputs).logits

                predicted_class_id = logits.argmax().item()
                print("True: ", label, "Pred: ", model.config.id2label[predicted_class_id])
                print("Text: ", text)
                cls_explainer(text)
                cls_explainer.visualize()
                print("----------------------------------------------------------------------")
        else:
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True, max_length=512)
            # tranformers interpret gives error for input of 512 tokens
            if len(inputs[0]) != 512:
                with torch.no_grad():
                    logits = model(**inputs).logits

                predicted_class_id = logits.argmax().item()
                print("True: ", label, "Pred: ", model.config.id2label[predicted_class_id])
                print("Text: ", text)
                cls_explainer(text)
                cls_explainer.visualize()
                print("----------------------------------------------------------------------")

In [17]:
scenarios = ["Speech", "Important_Event", "Cinderella", "Stroke", "Cat"]
# make sure not to interpret on trained scenarios
texts = load_from_disk(filepath + "/dataset")["test"].remove_columns(["Unnamed: 0", "input_ids", "attention_mask"]).to_pandas()
texts["label"] = [model.config.id2label[x] for x in texts["label"]]
n = 3

# Speech scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [18]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[0])].sample(n=n)])

In [19]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  working on it each day. it's better [FP] but i read each day. from the bible and the devotions and i from a book. i [FP] there's a book that has forty thousand words. and it three columns each page. and i read reach each one word. the the [FP] i i read the one column. and i read i read it twice. and i read and i read and i make a a a [FP] sentence a i make a sentence each one. and i i record it. and listen to it. and go on there. each day i go thirty three thirty minutes for that. it [FP] from the bible [FP] canterbury no large one. and each i i i read two pages each morning.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  and i'm gonna do and and [FP] able to converse with them more. and i think my speech will you know lift up compared to what it's like now. not good. i try. but no i i can real short stories i can do but something that takes a real long time i have a hard problem with. no.


----------------------------------------------------------------------
True:  ANOMIC Pred:  CONDUCTION
Text:  i don't know. but you know everybody says that they do something and i don't know it. that's what i do. so i don't know what i'm doing. oh it's been fine. mhm.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  ANOMIC
Text:  [FP] it it seems better [FP] until [FP] the first almost a first year was i said it happened me on august of a year ago. and [FP] [FP] after having been in the hospital for two weeks then i was at home by myself. and until i got into a group of people from [FP] people who were helping me with [FP] [FP] speaking [FP] i would it was difficult for me to to speak and talk english after all these years. no nine nine weeks. not weeks, months. from from august until [FP] i think it was april in this. [FP] apriata i didn't hafta pay for them. and there were two women. if it was three days a week. and [FP] either on of the woman would come in. and a lot of language of eating me and helping me with that. and [FP] [FP] [FP] so then that they felt that was in their probably. and then i to inch to take [FP] [FP] and auctioned it or [FP] [FP] to go to get [FP] because i had a car. you know i

----------------------------------------------------------------------
True:  CONDUCTION Pred:  CONDUCTION
Text:  for me that's like seventy percent eighty eighty percent surround. sometimes they're day [FP] some day badder sometimes better yeah.


...

----------------------------------------------------------------------
True:  CONTROL Pred:  ANOMIC
Text:  no i do not.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.16),CONTROL,-1.81,[CLS] no i do not . [SEP]
n/a,(0.79),ANOMIC,1.87,[CLS] no i do not . [SEP]
n/a,(0.57),CONDUCTION,1.47,[CLS] no i do not . [SEP]


----------------------------------------------------------------------
True:  CONTROL Pred:  ANOMIC
Text:  yes hindi.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.59),CONTROL,0.29,[CLS] yes hindi . [SEP]
n/a,(0.61),ANOMIC,0.62,[CLS] yes hindi . [SEP]
n/a,(0.29),CONDUCTION,-1.07,[CLS] yes hindi . [SEP]


----------------------------------------------------------------------


# Important event scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[1])].sample(n=n)])

In [ ]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  and he was the second oldest. he didn't get married until he was [FP] fifty or something you know. about [FP] a nineteen sixty somewhere around there. and it had a big v eight engine. and back then the cars had whopper big v eight engines. you know the standard engine. and it was like [FP] three ninety eight cubic inch. and it would do you know you could really get out on the highway. and and and if you didn't get caught by a policeman or something you know you'd be alright. clean you know. yeah. oh yeah.


----------------------------------------------------------------------
True:  ANOMIC Pred:  CONTROL
Text:  i useta speak spanish. no english. [FP] when i had my my first stroke i forgot all the spanish because i needed to concentrate on the english. so it just went out of out of it. [FP] i do a little spanish now but it's not fluent. you know i do try but sometimes it just doesn't come out proper. you know even with. if i'm upset or something i start you know the words don't come out. i just get more flustered. [FP] what else? i think the saddest day was probably when my father passed away. and that day i had gotten a raise from my job. and i was so depressed. six months later i had my stroke. and nobody really knows what happened. because now i'm heavy. before i was very slim. but he was you know the light of my life. and so when he passed away i just couldn't take. it it was just too much for me. and then i had my first stroke. and i then i would start thinking you know. i wish you w

----------------------------------------------------------------------
True:  ANOMIC Pred:  CONDUCTION
Text:  yeah i was happy as an adult. i i don't i don't know what it's called. yeah describe it. it's [UP] with letters. [FP] i describe with letters. it's a feeling. it's it's something to do with my life. that's is that's all they did. [FP] i don't know what i don't know what it is right now though. yeah. yeah. yes.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  CONDUCTION
Text:  mhm. mhm. mhm. mhm. yes yes happy happy yes. firstnamer hadta golf. firstnamer golf yes. five days a week golf. yes golf. yes yes numb numb tingly clumsy clumsy. no no no sorry. [FP] yes. yes fifty one yes fifty one years golf yes. every day golfing. four years four years back no golf. yes. [FP] five six i don't know. golf golf golf golf. move on move on move on. yes golf. projects projects yes house projects yes outside inside projects. [FP] happy yes happy. firstnamer motorcycle three times one week three times motorcycle. having fun. yes. [FP] sorry. yes. georgia yes georgia.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  CONDUCTION
Text:  long story. two years i lived with her, well my dad. a long time ago bellaire and edgewood. two years going in my house, bellaire. and gone, gone, gone. all the time gone, gone, gone. mhm.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  i guess that was an important day in my life. and [FP] i remember i was student teaching as a speech language pathologist. i was working hard. back then we didn't have super duper speech company so we hadta make everything. and i had contact paper that i was laminating pictures with. and my husband. now he's my husband but then he was my [FP] boyfriend. had called and said. oh let's go over to the park. let's do this. and he was acting kind of funny. and i thought. i don't really have time for this. and [FP] even when he came to the house he was acting kind of funny. and i said. oh you know i really wanted to finish what i was doing. so we went to brookdale park which is where we live right now across the street from brookdale park now. and [FP] that was the day that he asked me to get married to marry him. and then [FP] i was so shocked. and then i went home. and my my parents k

----------------------------------------------------------------------


# Cinderella scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[2])].sample(n=n)])

In [ ]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  and he married in a he he lived with his his mother inlaw with her two two girls. and she hadta work on there at the house. and [FP] and they she she hadta live in the coal bin. and they called her cinderella. and and and when her when her when [FP] her her girls they they were invited to a ball and and also their mother mother inlaw. and and she and she was she was she was also invited. but she's locked locked in the house. and they went there. but their their fairy fairy mother. she came here. and made her a dress for her. and and made a pumpkin pumpkin and a carriage with horse two horses. and [FP] there's some is who who is stood as a the man to takes takes him. and they went they they went to ball. and she met the prince. and [FP] and she her her fairy mother told her she hasta leave by midnight. and when he when when he when he when he heard the struck twelve. she ran and she left a class glass glass slipper. and and she ran home. and [FP] in th

----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  and the the girls were going but the the [FP] cinderella couldn't go because she didn't have anything to wear. she had cinderella [FP] they made she made [FP] cinderella some clothes to wear. well anyways said that she had hadta be home by [FP] twelve o'clock. and well and [FP] [FP] at the dance the [FP] girls were mad and the ugly mother were mad. and [FP] and and somebody tripped. oh no. she [FP] she hadta get hadta get home. and then they knocked her locked her up. and the little mice got the key and let her in out. and she [FP] and [FP] whoever he is [FP] looked at each shoe and found that the shoe belongs to cinderella. and the prince found her or [FP] they married and whatever. did i did i?


----------------------------------------------------------------------
True:  ANOMIC Pred:  CONTROL
Text:  and the stepsisters wasn't bitsy. [UP] but [UP] she was [UP] surprised when the prince announced the dance. she was very surprised when the stepsisters don't let her go. so they made her a dress. and [UP] fairy godmother makes her a shoes and wagon. no no. [UP] she goes to the dance. and [UP] the prince is quite taken with cinderella. the [UP] [FP] twelve 'col o'clock happened. and cinderella [UP] runs down the steps and [UP] loses one shoe. [UP]. [UP] the prince finds one shoe and [UP] says. i mutts find this woman. [UP] he was [UP] very confused by the woman [UP] who not fits the shoes. [UP] he [UP] he putts the chew on cinderella. and she fits the shoe. [UP] they were married. and that's the end.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  ANOMIC
Text:  i remember that. [FP] and she useta hadta do all the work. [FP] then [FP] there was going to be a ball. the mother let's say was favored the two of those so she would try to get them to go to the ball and stuff like that and put her to work you know. [FP] i know the mice came to make the dress. she got dressed. and the carriage came to take her to the ball. and the one thing that she told her is. make sure that you get home by midnight. [FP] and then it she will not no longer have the nice dress and all that. so [FP] and of course she met her prince or he liked her you know that type of thing at the time. and then she was gone. she she lost her shoe. so the prince was looking for the person who had that that matched with that her shoes or shoe. and [FP] let's see. they [FP] and of course the everybody went from house to house to house. and the two sisters of course were bigger 

----------------------------------------------------------------------
True:  CONDUCTION Pred:  ANOMIC
Text:  no. the [UP] magic. the [UP] the [UP] girls the one two [UP] [FP] [UP] cat and [UP] [FP] [UP] woman and can [FP] one two. the cat. welcome home. [FP] the one two the fan the fan. and. working hard. and the coffee to drink. one two. [FP] the kerchief. and hard hard. and [FP] the [FP] [UP] the mmice and two free and [UP] [FP] horse and [UP] dog and. yeah alright. cinderella. and gowns gowns. and and [FP] [FP] the poor cindy. and and [FP] [UP] mansion mansion. and. poor. the the book book and mansion mansion and the [FP] the [FP]. no. the gowns is beautiful. and and [FP]. i love you. and the and and [FP] doll and horse. and. i love her gowns. [FP] wearing the white gowns. the [FP] the prince the [FP]. sank you, thank you, thank you. the prince and [UP] the [FP] dancing dancing. the prince and cinderella and dancing. one one one. [FP] no. one [UP] [FP] hurried. hurry up! hurry up! 

----------------------------------------------------------------------
True:  CONDUCTION Pred:  CONDUCTION
Text:  you see you hate this so i can't. cannot see where where the bachelor. then she wore. i can't do. [FP] she he he falls. she brushes on by the. can't and she hates it. and thinks he's going to be house. and he can't. you see a beautiful view for miles. and then she runs. she hasta go home. come back out. he's like. oh i got one pair. and here i come. and he came home. and he did the prayer. and [FP] for the beautiful wedding work out. i can't do that. no. oh god.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  [FP] and there's also a pet cat and a very sweet dog. [FP] cinderella [FP] grows up to be a servant in the house. [FP] and her stepsitters stepsisters grow up as well only they're not as beautiful as cinderella is. cinderella [FP] [FP] works around the house. she's working in the stables [FP] in the shadow of the [FP] of the castle. [FP] one day cinderella's [FP] stepmom gets an invitation to the castle. there's gonna be a party. [FP] cinderella makes a really beautiful dress. [FP] and she gets ready. and she's ready to go with them. but the stepmom encourages her daughters who aren't very pretty to to destroy cinderella's dress. and now she can't go to the ball either. [FP] and then incredibly there's a [FP] fairy godmother who fixes cinderella's dress. who gives her [FP] a carriage with horses and a really beautiful dress. so cinderella goes to the ball. and she dances with the

----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  her mom had [FP] passed away. and her dad found a new wife who also had two daughters. so as they were blending these families together [FP] cinderella spent a lot of time with her stepmother. but her stepmother favored her children over cinderella and had cinderella do all the chores in their house. and the stepsisters didn't really have to do any chores. they were the favorite ones. but cinderella never complained. she made friends with all the animals around the house and was just a happygolucky girl. and one day there was going to be a big ball at the castle. and [FP] all the young maidens in the land were invited to attend the celebration. but cinderella's stepmother said she couldn't go because she did not have a proper dress. but her stepsisters were going to go. and they had the best dresses around. and [FP] cinderella found her trunk with a dress. but her mother said it 

----------------------------------------------------------------------


# Stroke scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    df = texts[(texts["label"] == x) & (texts["scenario"] == scenarios[3])]
    # no controls in stroke scenario
    if len(df) >= n:
        data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[3])].sample(n=n)])

In [ ]:
interpret(data)

True:  ANOMIC Pred:  CONDUCTION
Text:  [FP] [FP] june thirty thirtieth. [FP] let's see. not a thing. you know. but i it didn't know what heck is went wrong! i i i couldn't speak. and i couldn't my right leg and my right arm. you know i i just couldn't figure out what was why it what is wrong here you know? but [FP] man it's really weird you know. you know what? that i hadta pee. really. my my wife was there. wow. but anyway she gets off a little pee and you know. and then you know what? i i i thinking. i don't know where it comes out of! you know? you know i i i think my nose. and pee out there. i mean man i was going. oh. i'm in really trouble you know. yeah. [FP] real good. and the right arm not good at all you know. speech i hated it. it it doesn't do any good. then i came over here. janet whitehouse. and then i got finally apraxia. [FP] was the first thing i've gotta you know. [FP] i couldn't sound out words on the [FP] [FP] paper. and so i took two years to do that you know. and t

----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  [FP] one and a half days ago nothing. [FP] nothing. and [FP] following [FP] one week i c u slowly resolving. yeah i c u saint at saint joe's medical center towson maryland. [FP] floating in air. [FP] year and a half [FP] [FP] two and a half years resolved. [FP] i was at [FP] [FP] galveston texas. [FP] i was rehabbing at galveston texas [FP] [FP] [FP] tlc.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  oh we [FP] i had a stroke. and i was [FP] bed laid up about [FP] three or four or five days. and then i had some [FP] on the the the. but [FP] i [FP] and that's it. yes now for the recovery i have had about one two three four five six operations. [FP]. and a couple. and i did that by in about oh then twelve then for wait fifteen weeks weeks ago. and then [FP] i i i [FP] that and that. and i did the that. and i did i did a little bit of the that. but i could had [FP] [FP]. i could have lots of time. i did the homework and [FP] that for texas tech for about twelve to one fifteen months ago. and i had them. and and i or this had they have a a a system where i did [FP] a [FP] a [FP] stuck stroke [FP] a a a head heading or something. [FP] and i sksstk. and i have had about three three three or two deals with them. and i had [FP] good some they said i said. [FP] and i did [FP] extra extr

----------------------------------------------------------------------
True:  CONDUCTION Pred:  CONDUCTION
Text:  [FP] the hospital, nothing. bus the other place, kernan. the speech and all of that thing was berry good. kernan. i like kernan. no no long time ago. fourteen years ago. five. five and five. seven. ten years ago. [FP] i guess. uhhuh a long time ago. and now happy all the time. the stroke. no. angry person all the time. drinking a lot alone alone. not good. no no no. but two person drinking [FP] beer or two or three relax. no more alone. no good. uhhuh. [FP] all the time waving waving at everybody, waving. [FP] a wave at the people going in and that going away. hi hi. all the time hi.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  ANOMIC
Text:  and quarter after six. and donut i was [FP] trying to get [FP] coffee for firstnamer and i. it's with you. but anyway [FP] there was something that was wrong with the [FP] car. or there was something with me. and [FP] it it [FP] crossed the center line. and i said. we're in trouble now. this is it. so i turned off the key. and i and i cut it off. and i stopped right there in at the intersection. and [FP] a a car was was was police was was there coming my my way. and this guy was a is a knight. he knew. and [FP] he said get out of there. and he stopped [FP] he stopped there. and he said do it and try to try to get out of the car. and i did. i just opened the door. and i guess [FP] with the [FP] pavement. from there i've been in i've been in the hospital for three days. i i i think they'd say i'm in there for two two three days. and [FP] i came out once and twice with [FP] profan

----------------------------------------------------------------------
True:  CONDUCTION Pred:  CONDUCTION
Text:  but it's i think it's very slowly that it's getting a little bit better. i hope. and [FP] my brother was coming to come over to visit. and she was just gone me me. and all of a sudden that's i did that's that's all i know. i i i was i he he and and she wife came. they hadta win a cop and get it in to go with the line. and [FP] they hadta they he went over and went went to [UP] from the tea the the the light through the garage. i ended up in the hospital. that was it. it's it's nelf stj. from the. and [FP] speech filet obviously. and a lot of that because it's it's it's so [FP] it's very it takes a long of time. and that's that mainly i've that's mainly i've done that.


----------------------------------------------------------------------


# Cat scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[4])].sample(n=n)])

In [ ]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  [FP] dog barking dog barking. [FP] listened no listen. no. firemen come [UP] rescue the man.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.09),CONTROL,-4.08,[CLS] [FP] dog barking dog barking . [FP] listened no listen . no . fire ##men come [UP] rescue the man . [SEP]
n/a,(0.84),ANOMIC,4.10,[CLS] [FP] dog barking dog barking . [FP] listened no listen . no . fire ##men come [UP] rescue the man . [SEP]
n/a,(0.62),CONDUCTION,2.47,[CLS] [FP] dog barking dog barking . [FP] listened no listen . no . fire ##men come [UP] rescue the man . [SEP]


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  to go use the ladder the ladder to get on the cat foot to go down from the tree. so the father was on the tree to help the cat come down from the tree. [FP] [UP] this one the fire department come down to see [FP] the trees being [FP] fire department to go down and the bird to help themself from the fire department. this is the child comes to help also from the mother to helpful with pulling down take down the the the child not the child the cat. alright.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  dog is climbing up the tree. and [FP] he was [UP] barking his but he didn't know quite how to do it. and he's the cluster fire [FP]. damn that's not quite right. to [FP] take off the tree and you know. yeah.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  CONDUCTION
Text:  i've seen this before! i've seen this picture! you know me. i it's bad on me. so it's a girl. and this girl is [FP] a bicycle girl. she has a bicycle. and she's had her bicycle for herself. and she sees her cat. i i'm saying the right words? you got a cat. and the cat is up in the [FP] up in the wall. i'm saying the wrong word. but on this thing here the wall or whatever you call it. and and what's happening is she has a dog. and the dog is up on the tree. do i said the right word? the fence or tree. i guess a tree you call it. and her father a guy a guy is standing here. and he's got his hands on the [FP] deal. somehow he got up here. and he and he's got on the deal. and he's trying to get a chair the desk table. table i table l e b [FP] label table. and she wants to bring that table to bring it up here to put on here. and [FP] she got a couple of animal birds [FP] birds h

...

----------------------------------------------------------------------
True:  CONDUCTION Pred:  ANOMIC
Text:  well this is a little girl who has a cat. and the cat is up the tree. and [FP] i don't know if she was driving. she had her she was on a tricycle. so and here's a little the dog is up to is at the tree too. and he's barking. maybe that's what caused the [FP] the cat was afraid or something and went up there. either that or they're afraid of the sun. i've always said that as a joke. so so here's the ladder. now what it looks like the the [FP] maybe the ladder was up there. and it fell over. and so now the [FP] this is funny because now the this man maybe it's maybe it's her [FP] her father. because he's he's up there too. he's he's lost. he can't get down. and he's gonna perhaps he's gonna save both the [FP] his father and perhaps the cat too. and then there's a [FP] a bird up here [FP] who probably enjoys this whole thing. it's not gonna run away. now i'm. i was gonna say this 

----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  no no no no no no no. there was a dog who was chasing a cat up a tree. and [FP] so this man decided that he would try and get the cat outta the tree. but then he got stuck up the tree. and they hadta call the fire department to get not only him but the cat down from out of the tree.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  and the [FP] apparently a man has brought has noticed the activity going on and brought his ladder. and he's attempted to get the cat down. and he himself has gotten stuck in the tree while attempting to get the cat down. a dog may well have accompanied him. there was a dog there. and the [FP] fire department has also arrived with their ladder. and they are approaching the scene. and they are certainly going to get the cat down and well along with the man more than likely. and everything will be just fine.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  and her cat somehow got up in the tree. and [FP] the dog is barking because he wants the cat. [FP] and that's probably why the cat ran up the tree. and then a [FP] man climbed up in the tree to try to get the cat. and he can't get down. and the firefighters are coming to come rescue the man because his ladder fell down and to rescue the cat.


----------------------------------------------------------------------
